## This is a tutorial for doing colabfold interaction screening.

### Outline
* Prerequisites
* Collecting sequences
* Running mmseqs2 on monomers
* Renaming the output
* Creating the multimer msas
* Re-arranging the input
* Running colabfold batch on multimer msas
* Analyzing the output

### Prerequisites

This tutorial assumes that readers are familiar with linux, bash, slurm, and python.  Typical setup takes around a day and typical runtime is 1500 interactions per day on a NVIDIA A100 GPU.

### Collecting Sequences
Prior to running the workflows, you will need to have sequences in fasta file format (text file with alternating lines of >id and AA sequences with optional gaps in between records and line returns).  Query and bait sequences should be in separate files.  I would highly recommend running colabfold on all monomer sequences first.  Average plddt values for monomers below 70% are unlikely to yeild good results.  Note that ids will be used as filenames for the output so follow good linux file naming practices: no duplicates, no spaces (use _ instead), no special characters, ect...

### Running mmseqs2 on the monomers
The file [run_mmseqs2.sh](run_mmseqs2.sh) is an example slurm script running the mmseqs2 workflow.  It uses the colabfold_search command in the colabfold module to ensure appropriate formatting for colabfold input.  The command is as follows:
```bash
colabfold_search fasta_file.fa /n/data1/colabfold msas/
```
Here "fasta_file.fa" is the input sequence file, /n/data1/colabfold is the location of the databases (may be different in your environment), and msas/ is the folder where the output will be placed.  

Start by creating a linux accessible project folder and putting the fasta files in it.

Note that msas must be pre-created and have execute permissions enabled.  Here is how to make it and set the permissions:
```bash
mkdir msas
chmod -R +x msas
```
Use *ls -l* to check that the execute permissions have been set appropriately.  Then make a copy of the example slurm script mentioned above in your project folder.  Change the output log file and command line to the desired names.  Then submit the job to slurm with sbatch run_mmseqs2.sh.  You can monitor the progress by checking the log and err files and monitoring slurm with *sqeue*.

You will want to edit the above file and rerun the job for the bait sequences, specifying a different msa folder.  This job is set up to use quite a bit of cluster resources so it is recommended to either run the jobs one at a time or check node availability (with *sinfo*) before submitting both jobs.  Make sure that all scripts reference your new msa folder name and it has execute permissions set before running it.

### Renaming the output
Mmseqs2 names the files according to their numerical order in the input file.  You will use the script [rename_msas.py](rename_msas.py) to rename them.  First start an interactive session with *sinteractive*.  Then load the python module with *module load python* (or the equivalent module or mamba environment on your system).  Then run the script as follows:

```bash
python rename_msas.py --fastafile fasta_file.fa --msadir msas/
```

Note that you can run all of this non-interactively in an sbatch script if desired but it doesn't take very long or use significant resources.

### Creating the multimer msas
Next we merge the query and bait msas into multimer msas that will be input into colabfold for prediction.  This process is done with a python file called [create_msas_multimer.py](create_msas_multimer.py).  Because it can be fairly time consuming, I created a slurm script to run it: [create_msas_multimer.sh](create_msas_multimer.sh).  Note that this script relies on my jpdbtools2.py script--that path needs to be changed in the python script before running.  The command in the shell script that does the work is this:
```bash
python create_msas_multimer.py --msa1_dir query_msas/ --msa2_dir bait_msas/ --out_dir interaction_msas/
```
You will need to make sure create_msas_multimer.py is in your project directory and that the query_msas and bait_msas folders match the desired names.  Run it with slurm and monitor as usual.  There is an optional parameter called --limitlen that will cap each msa list at the maximum number of msas.  This will reduce colabfold memory requirements but may also reduce accuracy.  This script creates query x bait number of outputs so it can take significant time to run and can easily create hundreds of thousands of files.  The output file names will be queryname_baitname_#.a3m.  The # value is unimportant.

### Re-arranging the output (optional)
At this point we likely have an interaction_msas folder with a gigantic number of interaction msas.  The create_msas_multimer script puts things in subfolders so if you have less than 500 msas in each subfolder skip this step.  To make it easier to run colabfold on these, it is best to rearrange those.  The easiest way is to put each set of query msas in it's own folder.  I have a simple python script written for that called [separate_multimer_msas.py](separate_multimer_msas.py).  Load the python module in an interactive session and run it as follows:

```bash
python separate_multimer_msas.py --dir interaction_msas/
```

The result is that all of the interaction msas will be moved into query subfolders.

### Running colabfold batch
We're finally ready to run the batch interaction prediction!  Assuming you've put your interaction msas into subfolders as specified above, you can run colabfold as an array job with [run_cfb_msa_multimer.sh](run_cfb_msa_multimer.sh).  That script has a list of folder names (default is "query1,...") that need to correspond to our interaction_msas folder names--if they don't you should edit them to match.  Here is the key command in that file:
```bash
colabfold_batch --model-type alphafold2_multimer_v3 --num-models 1 --num-recycle 3 interaction_msas/${querylist[$selidx]} outputdir_multimer_msas/${querylist[$selidx]}/
```
Here alphafold2_multimer_v3 is the model we are using for prediction.  The *--num-models 1* says that we will create one model per prediction.  Multiple models would take at least twice as long and don't seem to impact the interaction metrics much (they mostly impact the predicted structure) so we don't use them here.  The *--num-recycle* parameter is similar.  Larger recycles typically indicate a poor prediction which we aren't interested in anyway and would slow our predictions.  Run the script with sbatch and monitor output as before.  The outputdir_multimer_msas subfolders will each have a log.txt file in them monitoring the predictions.  There will also be colabfold_msa_multimer_job_array.log and .err files output that follow output and errors for each array task.  Note that if there is GPU availability, you can change the *--array=1-17%1* % value to use more GPU's.  For initial runs I would recommend only using one GPU to avoid overusing the cluster.

### Analyzing the output
Let's explore the results a bit.  The outputdir_multimer_msas folder will have 8 outputs for each interaction msa.  Each prediction starts with the a3m file name (should be query_bait.a3m) and ends with a suffix.  Here is a list of those files and what they describe:

| File Suffix | Description |
|:------------|:------------|
| .a3m | Copy of the a3m file (exact?) |
| \_coverage.png | Picture of the alignment coverage map |
| \_pae | Picture of the predicted aligned error (pAE) matrix |
| .done.txt | Empty file indicating prediction is complete |
| \_plddt.png | Picture of the plddt plot (query then bait) |
| \_predicted_aligned_error_v1.json | A json pAE matrix |
| \_scores_rank_001_alphafold2_multimer_v3_model_1_seed_000.json | A json dictionary with pAE, plddt, pTM, and ipTM scores |
| \_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb | A pdb file with the predicted structure |

From here on out the analysis is focused on the "scores" file or the pdb file.  The pdb file is only used to get protein length information but could be used for more complex analysis.  The scores file is a json dictionary with entries for *pae*, *plddt*, *ptm*, and *iptm*.  The pae entry is a json list matrix (list of lists) with an entry for each pair of amino acids.  The plddt entry is a simple list with a plddt score for each amino acid.  The ptm and iptm entries are simple numerical values.  The python *json* module allows for simple reading of json files into a dictionary structure.  You can get the outputs as numbers or numpy arrays as follows:

```python
import json
import numpy as np

with open('filename.json','r') as f:
    jdict=json.load(f)
    
paemat=np.array(jdict['pae'])
plddt=np.array(jdict['plddt'])
ptm=jdict['ptm']
iptm=jdict['iptm']
```

At this point it is useful to know the lengths of the query and bait so that you can more easily interpret the pAE and plddt information.  In principle you can get these from the msa or original fasta files but I'm lazy and prefer to read in the pdb file.  I have a module (/n/projects/jru/public/IPython_Notebooks/pdbtools/jpdbtools2.py) that will read the pdb file into a dataframe for processing as follows:

```python
import sys
sys.path.append('/n/projects/jru/public/IPython_Notebooks/pdbtools/')
import jpdbtools2 as jpt
import pandas as pd

#read the pdb into a dataframe
pdbdf=jpt.cleanpdbdf(jpt.getpdbdf('filename.pdb'))
#now get the protein lengths--the last residue should be the bait length
baitlen=pdbdf.iloc[-1]['residue']
totlen=len(paemat)
querylen=totlen-baitlen
```

At this point the average plddt calculation is academic with simple numpy codes:
```python
plddt1=plddt[:querylen].mean()
plddt2=plddt[querylen:].mean()
```

The pAE matrix can be a bit trickier to work with so here is a function that will parse it out:
```python
def getPAEInteraction(paemat,len1,len2):
    '''
    get's the predicted aligned error for the interaction and each protein
    '''
    #the pae interaction value is the mean of the off-diagonal blocks
    paei=0.5*(paemat[:len1,len1:].mean()+paemat[len1:,:len1].mean())
    #the query and bait pae values are the means of the on-diagonal blocks
    pae1=paemat[:len1,:len1].mean()
    pae2=paemat[len1:,len1:].mean()
    return paei,pae1,pae2
```

From here on out the analysis is standard data science.  You will probably want to put all of your metrics in lists and then in a dataframe for sorting and filtering and export.  The [evaluate_interactome.ipynb](evaluate_interactome.ipynb) file shows an example of this for a list of peptides binding to a bacterial proteome.

Happy Interactome Predicting!